In [13]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "worldbank_data_exploration"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

In [14]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

from data_sources.get import get_indicators

In [15]:
df = get_indicators(
    [
        "SP.POP.GROW",
        "FP.CPI.TOTL.ZG",
        "SP.DYN.LE00.IN",
        "NE.EXP.GNFS.ZS",
        "NY.GDP.MKTP.KD.ZG",
        "SL.UEM.TOTL.ZS",
        "NV.AGR.TOTL.ZS",
        "EG.ELC.ACCS.ZS",
        "AG.LND.FRST.ZS",
        "SH.DYN.MORT",
        "NY.GDP.TOTL.RT.ZS",
        "SP.DYN.TFRT.IN",
        "EN.URB.LCTY.UR.ZS",
        "TG.VAL.TOTL.GD.ZS",
        "MS.MIL.XPND.GD.ZS",
    ]
)

print(df["Indicator Name"].unique())

['Population growth (annual %)' 'Inflation, consumer prices (annual %)'
 'Life expectancy at birth, total (years)'
 'Exports of goods and services (% of GDP)' 'GDP growth (annual %)'
 'Unemployment, total (% of total labor force) (modeled ILO estimate)'
 'Agriculture, forestry, and fishing, value added (% of GDP)'
 'Access to electricity (% of population)' 'Forest area (% of land area)'
 'Mortality rate, under-5 (per 1,000 live births)'
 'Total natural resources rents (% of GDP)'
 'Fertility rate, total (births per woman)'
 'Population in the largest city (% of urban population)'
 'Merchandise trade (% of GDP)' 'Military expenditure (% of GDP)']


In [16]:
df = df.pivot_table(
    values="Value", index="Year", columns=["Indicator Name", "Country Name"]
)

In [17]:
df_nans = df.loc[slice(2000, 2020)].isnull().sum().reset_index()

In [18]:
aggregates = np.array(
    [
        "Africa Eastern and Southern",
        "Africa Western and Central",
        "Arab World",
        "Caribbean small states",
        "Central Europe and the Baltics",
        "Early-demographic dividend",
        "East Asia & Pacific",
        "East Asia & Pacific (excluding high income)",
        "East Asia & Pacific (IDA & IBRD countries)",
        "Euro area",
        "Europe & Central Asia",
        "Europe & Central Asia (excluding high income)",
        "Europe & Central Asia (IDA & IBRD countries)",
        "European Union",
        "Fragile and conflict affected situations",
        "Heavily indebted poor countries (HIPC)",
        "High income",
        "IBRD only",
        "IDA & IBRD total",
        "IDA blend",
        "IDA only",
        "IDA total",
        "Late-demographic dividend",
        "Latin America & Caribbean",
        "Latin America & Caribbean (excluding high income)",
        "Latin America & the Caribbean (IDA & IBRD countries)",
        "Least developed countries: UN classification",
        "Low & middle income",
        "Low income",
        "Lower middle income",
        "Middle East & North Africa",
        "Middle East & North Africa (excluding high income)",
        "Middle East & North Africa (IDA & IBRD countries)",
        "Middle income",
        "North America",
        "OECD members",
        "Other small states",
        "Pacific island small states",
        "Post-demographic dividend",
        "Pre-demographic dividend",
        "Small states",
        "South Asia",
        "South Asia (IDA & IBRD)",
        "Sub-Saharan Africa",
        "Sub-Saharan Africa (excluding high income)",
        "Sub-Saharan Africa (IDA & IBRD countries)",
        "Upper middle income",
        "World",
    ]
)

aggregates.shape

(48,)

In [19]:
countries_with_nans = df_nans[df_nans[0] > 2]["Country Name"].unique()
countries_with_nans

array(['Afghanistan', 'Algeria', 'Cabo Verde', 'Congo, Rep.',
       'Equatorial Guinea', 'Georgia', 'Guinea-Bissau', 'Iraq',
       'Kiribati', "Korea, Dem. People's Rep.", 'Lebanon', 'Liberia',
       'Malaysia', 'Myanmar', 'Sierra Leone', 'Somalia', 'South Sudan',
       'Syrian Arab Republic', 'Turkey', 'Tuvalu', 'Armenia', 'Aruba',
       'Bahrain', 'Barbados', 'Bermuda', 'Cayman Islands',
       'Central African Republic', 'Djibouti', 'Eritrea',
       'French Polynesia', 'Greenland', 'Isle of Man', 'Kosovo', 'Libya',
       'Liechtenstein', 'Maldives', 'Nauru', 'New Caledonia',
       'San Marino', 'Sint Maarten (Dutch part)', 'Solomon Islands',
       'St. Lucia', 'Venezuela, RB', 'Curacao', 'Dominica', 'Ethiopia',
       'Guyana', 'Lao PDR', 'Lesotho', 'Northern Mariana Islands',
       'Papua New Guinea', 'Suriname', 'Virgin Islands (U.S.)',
       'Yemen, Rep.', 'Andorra', 'Marshall Islands', 'Palau',
       'St. Kitts and Nevis', 'Sudan', 'World', 'American Samoa',
       '

In [20]:
df_cleared = df.loc[2000:2020].stack()
print(df_cleared.shape)
df_cleared.drop(index=countries_with_nans, level=1, inplace=True)
print(df_cleared.shape)
df_cleared.drop(index=aggregates, level=1, inplace=True)
print(df_cleared.shape)

(5565, 15)
(3528, 15)
(2646, 15)


In [21]:
df_cleared = df_cleared.unstack()
df_cleared.bfill(inplace=True)
df_cleared.ffill(inplace=True)
df_cleared = df_cleared.stack()
df_cleared.dropna(axis=0, inplace=True)

In [22]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_cleared.values)
df_scaled = pd.DataFrame(
    data=data_scaled, index=df_cleared.index, columns=df_cleared.columns
)
df_scaled.head()

Indicator Name     Access to electricity (% of population)  \
Year Country Name                                            
2000 Albania                                      0.544732   
     Angola                                      -2.588390   
     Australia                                    0.544732   
     Austria                                      0.544732   
     Azerbaijan                                   0.501973   

Indicator Name     Agriculture, forestry, and fishing, value added (% of GDP)  \
Year Country Name                                                               
2000 Albania                                                1.576034            
     Angola                                                -0.400709            
     Australia                                             -0.668088            
     Austria                                               -0.821985            
     Azerbaijan                                             0.691981            

Indicator Name     Exports of goods and services (% of GDP)  \
Year Country Name                                             
2000 Albania                                      -0.763197   
     Angola                                        2.018710   
     Australia                                    -0.791075   
     Austria                                       0.165737   
     Azerbaijan                                    0.038761   

Indicator Name     Fertility rate, total (births per woman)  \
Year Country Name                                             
2000 Albania                                      -0.275781   
     Angola                                        3.022029   
     Australia                                    -0.570833   
     Austria                                      -0.862205   
     Azerbaijan                                   -0.391300   

Indicator Name     Forest area (% of land area)  GDP growth (annual %)  \
Year Country Name                                                        
2000 Albania                          -0.143781               0.874516   
     Angola                            1.661105              -0.094674   
     Australia                        -0.719076               0.118960   
     Austria                           0.825779              -0.014705   
     Azerbaijan                       -0.993436               1.909004   

Indicator Name     Inflation, consumer prices (annual %)  \
Year Country Name                                          
2000 Albania                                   -0.467139   
     Angola                                    28.546044   
     Australia                                 -0.073618   
     Austria                                   -0.262241   
     Azerbaijan                                -0.310443   

Indicator Name     Life expectancy at birth, total (years)  \
Year Country Name                                            
2000 Albania                                      0.184969   
     Angola                                      -3.222392   
     Australia                                    0.840674   
     Austria                                      0.703138   
     Azerbaijan                                  -0.708325   

Indicator Name     Merchandise trade (% of GDP)  \
Year Country Name                                 
2000 Albania                          -0.688447   
     Angola                            1.431515   
     Australia                        -0.852002   
     Austria                           0.155866   
     Azerbaijan                       -0.258386   

Indicator Name     Military expenditure (% of GDP)  \
Year Country Name                                    
2000 Albania                             -0.418478   
     Angola                               2.809607   
     Australia                           -0.052478   
     Austria                             -0.585358   
     Azerbaijan                           0.222279   

I

In [23]:
from data_sources.load_dataset import load_time_series, load_dataset

df = load_dataset()
print(df.isnull().sum().sum())
df

0


Indicator Name       Access to electricity (% of population)  \
Year Country Name                                              
2000 Albania                                      100.000000   
     Angola                                        20.000000   
     Australia                                    100.000000   
     Austria                                      100.000000   
     Azerbaijan                                    98.908222   
...                                                      ...   
2020 Uganda                                        41.300000   
     Ukraine                                      100.000000   
     United Kingdom                               100.000000   
     United States                                100.000000   
     Uruguay                                       99.900000   

Indicator Name       Agriculture, forestry, and fishing, value added (% of GDP)  \
Year Country Name                                                                 
2000 Albania                                                 24.515412            
     Angola                                                   5.664690            
     Australia                                                3.114903            
     Austria                                                  1.647300            
     Azerbaijan                                              16.084865            
...                                                                ...            
2020 Uganda                                                  23.929034            
     Ukraine                                                  9.266236            
     United Kingdom                                           0.581796            
     United States                                            0.916865            
     Uruguay                                                  7.481413            

Indicator Name       Exports of goods and services (% of GDP)  \
Year Country Name                                               
2000 Albania                                        20.115286   
     Angola                                         89.685831   
     Australia                                      19.418124   
     Austria                                        43.346274   
     Azerbaijan                                     40.170831   
...                                                       ...   
2020 Uganda                                         15.414799   
     Ukraine                                        39.061358   
     United Kingdom                                 28.120834   
     United States                                  10.134143   
     Uruguay                                        25.373271   

Indicator Name       Fertility rate, total (births per woman)  \
Year Country Name                                               
2000 Albania                                            2.157   
     Angola                                             6.639   
     Australia                                          1.756   
     Austria                                            1.360   
     Azerbaijan                                         2.000   
...                                                       ...   
2020 Uganda                                             4.824   
     Ukraine                                            1.228   
     United Kingdom                                     1.650   
     United States                                      1.705   
     Uruguay                                            1.963   

Indicator Name       Forest area (% of land area)  GDP growth (annual %)  \
Year Country Name                                                          
2000 Albania                            28.076642               6.946217   
     Angola                             62.331443               3.054624   
     Australia                          17.158156               3.912428   
     Austria   